In [1]:
!pip install evaluate

In [2]:
!pip install transformers

In [3]:
import numpy as np
import evaluate
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [5]:
my_data=np.genfromtxt('/gdrive/My Drive/kan-eng/spa-eng(2)/spa.txt',delimiter="\t",dtype=None)

<ipython-input-5-9cd0e085fe6b>:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  my_data=np.genfromtxt('/gdrive/My Drive/kan-eng/spa-eng(2)/spa.txt',delimiter="\t",dtype=None)


In [6]:
my_data

array([['Go.', 'Ve.', 'CC-BY 2.0 (France) Attribution: tatoeba.org'],
       ['Go.', 'Vete.', 'CC-BY 2.0 (France) Attribution: tatoeba.org'],
       ['Go.', 'Vaya.', 'CC-BY 2.0 (France) Attribution: tatoeba.org'],
       ...,
       ['If you want to sound like a native speaker, you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo.',
        'Si quieres sonar como un hablante nativo, debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un músico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado.',
        'CC-BY 2.0 (France) Attribution: tatoeba.org'],
       ['It may be impossible to get a completely error-free corpus due to the nature of this kind of collaborative effort. However, if we encourage members to contribute sentences in

In [7]:
eng_sentence=[]
spa_translation=[]
for pairs in my_data:
  eng_sentence.append(pairs[0])
  spa_translation.append(pairs[1])


In [8]:
!pip install sentencepiece

In [9]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")
model=AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-en")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [10]:
spa_translation[0],eng_sentence[0]

('Ve.', 'Go.')

In [11]:
model.to('cuda')

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,),

In [16]:
num_of_examples=5
inputs=tokenizer(
    spa_translation[:num_of_examples],
    truncation=True,
    padding=True,
    return_tensors='pt').to('cuda')
translation=model.generate(**inputs)


In [17]:
pretrained_output=[]
for t in translation:
  pretrained_output.append(tokenizer.decode(t,skip_special_tokens=True))

In [18]:
pretrained_output

['Go on.', 'Go away.', 'Wow.', 'Go away.', 'Hey.']

In [19]:
predictions = pretrained_output
references=[[sent] for sent in eng_sentence[:num_of_examples]]
bleu=evaluate.load("bleu")
results=bleu.compute(predictions=predictions,references=references)
print(results)

{'bleu': 0.0, 'precisions': [0.6153846153846154, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.3, 'translation_length': 13, 'reference_length': 10}
